# Create Containerlab topology from CYJS graph

Initialize parameters   

In [276]:
topology_name = "DM-Akron"

Read CYJS graph data into a dictonary and initialize networkx graph from it

In [277]:
import json
import networkx as nx

cyjs = {}
with open(topology_name + ".cyjs", 'r', encoding='utf-8') as f:
    cyjs = json.load(f)
G = nx.cytoscape_graph(cyjs)
print(G.graph)

{'name': 'DM-Akron'}


Parse graph G into lists of: nodes and links. Keep a list of interfaces per device in `device_interfaces`.

In [278]:
nodes, links = [], []
device_interfaces = {}
for n in G.nodes:
    if G.nodes[n]['type'] == 'device':
        dev = G.nodes[n]['device']
        nodes.append(dev)
        device_interfaces[n] = []
    elif G.nodes[n]['type'] == 'interface':
        int_name = G.nodes[n]['interface']['name']
        dev_name, dev_node_id = None, None
        peer_name, peer_dev_name, peer_dev_node_id = None, None, None
        for a_adj in G.adj[n].items():
            if G.nodes[a_adj[0]]['type'] == 'device':
                dev_name = G.nodes[a_adj[0]]['device']['name']
                dev_node_id = G.nodes[a_adj[0]]['device']['node_id']
                device_interfaces[a_adj[0]].append(int_name)
            elif G.nodes[a_adj[0]]['type'] == 'interface' and G.nodes[n]['side'] == 'a':
                peer_name = G.nodes[a_adj[0]]['interface']['name']
                for b_adj in G.adj[a_adj[0]].items():
                    if G.nodes[b_adj[0]]['type'] == 'device':
                        peer_dev_name = G.nodes[b_adj[0]]['device']['name']
                        peer_dev_node_id = G.nodes[b_adj[0]]['device']['node_id']
        if G.nodes[n]['side'] == 'a':
            links.append({
                'a': {
                    'node': dev_name,
                    'node_id': dev_node_id,
                    'interface': int_name,
                },
                'b': {
                    'node': peer_dev_name,
                    'node_id': peer_dev_node_id,
                    'interface': peer_name,
                },
            })

print(nodes)
print(device_interfaces)
print(links)

[{'id': 1, 'type': 'device', 'name': 'dmi01-akron-rtr01', 'node_id': 0}, {'id': 14, 'type': 'device', 'name': 'dmi01-akron-sw01', 'node_id': 5}]
{0: ['GigabitEthernet0/1/0', 'GigabitEthernet0/1/1'], 5: ['GigabitEthernet1/0/1', 'GigabitEthernet1/0/2']}
[{'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/0'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/1'}}, {'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/1'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/2'}}]


Create container-compatible interface names for each device. We assume interface with index `0` is reserved for management, and start with `1`

In [279]:
c_device_interfaces = {}
for node_id, interfaces in device_interfaces.items():
    interfaces.sort()
    c_device_interfaces[node_id] = [f"eth{interfaces.index(i)+1}" for i in interfaces]

for l in links:
    a_idx = device_interfaces[l['a']['node_id']].index(l['a']['interface'])
    b_idx = device_interfaces[l['b']['node_id']].index(l['b']['interface'])
    l['a']['c_interface'] = c_device_interfaces[l['a']['node_id']][a_idx]
    l['b']['c_interface'] = c_device_interfaces[l['b']['node_id']][b_idx]

print(links)

[{'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/0', 'c_interface': 'eth1'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/1', 'c_interface': 'eth1'}}, {'a': {'node': 'dmi01-akron-rtr01', 'node_id': 0, 'interface': 'GigabitEthernet0/1/1', 'c_interface': 'eth2'}, 'b': {'node': 'dmi01-akron-sw01', 'node_id': 5, 'interface': 'GigabitEthernet1/0/2', 'c_interface': 'eth2'}}]


Generate clab topology. Using this gist as inspiration https://gist.github.com/renatoalmeidaoliveira/fdb772a5a02f3cfc0b5fbe7e8b7586a2

In [280]:
topology = {
    'name': G.name,
    'nodes': [f"{n['name']}" for n in nodes],
    'links': [f"[\"{l['a']['node']}:{l['a']['c_interface']}\", \"{l['b']['node']}:{l['b']['c_interface']}\"]" for l in links],
}

print(json.dumps(topology, indent=4))

{
    "name": "DM-Akron",
    "nodes": [
        "dmi01-akron-rtr01",
        "dmi01-akron-sw01"
    ],
    "links": [
        "[\"dmi01-akron-rtr01:eth1\", \"dmi01-akron-sw01:eth1\"]",
        "[\"dmi01-akron-rtr01:eth2\", \"dmi01-akron-sw01:eth2\"]"
    ]
}


Load Jinja2 template for Containerlab to run the topology through

In [281]:
from jinja2 import Environment, FileSystemLoader
env = Environment(
            loader=FileSystemLoader(f"."),
            line_statement_prefix='#'
        )
templ = env.get_template(f"clab.j2")

Run the topology through jinja2 template to get the final result

In [282]:
config = templ.render(topology)
print(config)

name: DM-Akron
topology:
    nodes:
        dmi01-akron-rtr01:
            kind: ceos
            image: ceos
        dmi01-akron-sw01:
            kind: ceos
            image: ceos
    links:
        - endpoints: ["dmi01-akron-rtr01:eth1", "dmi01-akron-sw01:eth1"]
        - endpoints: ["dmi01-akron-rtr01:eth2", "dmi01-akron-sw01:eth2"]

